In [ ]:
# imports
import numpy as np 
import numpy.random as random
import matplotlib.pyplot as plt
import math

In [ ]:
# Escape Probability Analytical
# t optical depth

def p_esc_analytic(t):
  return (3. / (4. * t) * (1. - 1. / (2. * t**2) + (1. / t + 1. / (2. * t**2)) * np.exp(-2. * t))) # probability

In [ ]:
class homogeneous_sphere_esc_abs(object):
    """Homogeneous Sphere class
    This class defines a homogeneous sphere with a specified total optical
    depth and performs a simple MCRT simulation to determine the escape
    probability. This is done automatically during the initialization step. The
    escape probability can be accessed via the class attribute p_esc. The
    effect of isotropic scattering can be included by setting albedo > 0. This
    parameter describes the scattering probability with respect to the total
    (i.e. scattering + absorption) interaction probability.
    Note: the analytic prediction p_esc_analytic only applies for albedo = 0,
    i.e. in the absence of scattering
    Parameters
    ----------
    tau : float
        total optical depth of the homogeneous sphere
    albedo : float
        ratio of scattering to total interaction probability (default 0.1)
    N : int
        number of MC packets that are setup up (default 10000)
    Attributes
    ----------
    p_esc : float
        escape probability as determined by MCRT
    """
    def __init__(self, tau, albedo=0.1, N=10000):

        self.RNG = random.RandomState(seed=None)
        self.N = N
        self.tau_sphere = tau
        self.albedo = albedo

        # initial position of packets in optical depth space
        self.tau_i = self.tau_sphere * (self.RNG.rand(self.N))**(1./3.)
        # initial propagation direction
        self.mu_i = 2 * self.RNG.rand(self.N) - 1.

        # number of escaping packets
        self.N_esc = 0
        # number of active packets
        self.N_active = self.N

        # perform propagation
        self._propagated = False
        self._propagate()

    @property
    def p_esc(self):
        """escape probability"""
        return self.N_esc / float(self.N)

    def _propagate(self):
        """Perform propagation of MC packets
        All packets are followed until they are absorbed or escape from the
        sphere.
        """

        if self._propagated:

            print("Propagation has already been performed!")
            print("_propagate call will have no effect")
            return False

        i = 0
        while self.N_active > 0:
            self._propagate_step()
            i = i + 1
            if i > 1e6:
                print("Safety exit")
                print("Propagation steps limit of {:d} exceeded".format(i))
                return False
        #print("Performed {:d} propagation steps".format(i))
        return True

    def _propagate_step(self):
        """Perform one propagation step
        All active packets are propagated to the next event which can either be
        a physical interaction or escaping from the sphere. If scatterings are
        active, it is decided for each interacting packet whether it is
        absorbed or scattered. All packets that are absorbed or escape during
        the current step are removed from the active pool.
        """

        # optical depth to next interaction
        self.tau = -np.log(self.RNG.rand(self.N_active))
        # optical depth to sphere edge
        self.tau_edge = np.sqrt(self.tau_sphere**2 - self.tau_i**2 *
                                (1. - self.mu_i**2)) - self.tau_i * self.mu_i

        # identify packets that escape
        self.esc_mask = self.tau_edge < self.tau
        # update number of escaping packets
        self.N_esc += self.esc_mask.sum()

        # identify interacting packets
        self.nesc_mask = np.logical_not(self.esc_mask)

        # decide which interacting packets scatter and which get absorbed
        self.abs_mask = self.RNG.rand(self.nesc_mask.sum()) >= self.albedo
        self.scat_mask = np.logical_not(self.abs_mask)

        # select properties of scattering packets
        self.tau = self.tau[self.nesc_mask][self.scat_mask]
        self.tau_i = self.tau_i[self.nesc_mask][self.scat_mask]
        self.mu_i = self.mu_i[self.nesc_mask][self.scat_mask]

        # update number of active packets
        self.N_active = self.scat_mask.sum()

        # update properties (position in optical depth space, propagation
        # direction) of scattering packets
        self.tau_i = np.sqrt(self.tau_i**2 + self.tau**2 +
                             2. * self.tau * self.tau_i * self.mu_i)
        self.mu_i = 2 * self.RNG.rand(self.N_active) - 1.

escape_prop = []
escape_prop_analytical = []
a = np.linspace(0.01,100,10000)
for i in a:
  mcrt_esc_prop = homogeneous_sphere_esc_abs(i)
  print("tau: {:.4e}, escape probability: {:.4e}".format(mcrt_esc_prop.tau_sphere, mcrt_esc_prop.p_esc))
  escape_prop.append(mcrt_esc_prop.p_esc)
  escape_prop_analytical.append(p_esc_analytic(i))

albedo_set = [0,0.1,0.5,0.95]
tau_set = [0.01,0.1,0,10,100]
results = []

for j in albedo_set:
  for i in tau_set:
    new_esc = homogeneous_sphere_esc_abs(tau = i, albedo = j)
    results.append((i,new_esc.p_esc))


Streaming output truncated to the last 5000 lines.
tau: 5.0010e+01, escape probability: 1.7600e-02
tau: 5.0020e+01, escape probability: 1.8800e-02
tau: 5.0030e+01, escape probability: 1.7500e-02
tau: 5.0040e+01, escape probability: 1.5600e-02
tau: 5.0050e+01, escape probability: 1.6100e-02
tau: 5.0060e+01, escape probability: 1.3100e-02
tau: 5.0070e+01, escape probability: 1.4600e-02
tau: 5.0080e+01, escape probability: 1.6600e-02
tau: 5.0090e+01, escape probability: 1.4100e-02
tau: 5.0100e+01, escape probability: 1.3900e-02
tau: 5.0110e+01, escape probability: 1.6600e-02
tau: 5.0120e+01, escape probability: 1.6900e-02
tau: 5.0130e+01, escape probability: 1.6700e-02
tau: 5.0140e+01, escape probability: 1.4800e-02
tau: 5.0150e+01, escape probability: 1.4900e-02
tau: 5.0160e+01, escape probability: 1.6200e-02
tau: 5.0170e+01, escape probability: 1.7000e-02
tau: 5.0180e+01, escape probability: 1.7700e-02
tau: 5.0190e+01, escape probability: 1.6800e-02
tau: 5.0200e+01, escape probability: 

In [ ]:
results
ok = []
b = []

for i in results:
  ok.append(i[0])
  b.append(i[1])

<Figure size 1440x864 with 0 Axes>

<Figure size 1440x864 with 0 Axes>

In [ ]:
# Plots
plt.plot(np.log(a),escape_prop_analytical, color = 'g', label = 'Analytical')

plt.title('Escape probability with different tau')
plt.xlabel('Tau')
plt.ylabel('P_esc')


plt.scatter([-4,-2,0,2,4],b[0:5],color = 'blue',s=15,label = 'X0/Xtot = 0')
plt.scatter([-4,-2,0,2,4],b[5:10],color = 'red',s=15, label = 'X0/Xtot = 0.1')
plt.scatter([-4,-2,0,2,4],b[10:15],color = 'orange',s=15,label = 'X0/Xtot = 0.5')
plt.scatter([-4,-2,0,2,4],b[10:15],color = 'purple',s=15, label = 'X0/Xtot = 0.95')
plt.legend()
plt.xticks([-4,-2,0,2,4],[0.01,0.1,1,10,100])
plt.figure(figsize=(20,12))